In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [13]:
nvi_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/Temp_NVIZones2026v2__20250226/Temp_NVIZones2026v2__20250226.shp")
detroit_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/City_of_Detroit_Boundary/City_of_Detroit_Boundary.shp")
cd_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/Detroit_City_Council_Districts_2026/Detroit_City_Council_Districts_2026.shp")

In [3]:
# Load birth data from csv
births_df = pd.read_csv('Q:/Vital Records/2023/2023 Birth Files.csv', low_memory=False)

In [4]:
# Assuming the birth data contains 'latitude' and 'longitude' columns
# Convert to a GeoDataFrame
births_gdf = gpd.GeoDataFrame(
    births_df, 
    geometry=gpd.points_from_xy(births_df.LONGITUDE, births_df.LATITUDE), 
    crs="EPSG:4326")

### Detroit - City Wide

In [7]:
# Ensure both GeoDataFrames have the same CRS
births_gdf = births_gdf.to_crs(detroit_shp.crs)

# Perform a spatial join to assign births to geographic areas
births_with_geography = gpd.sjoin(births_gdf, detroit_shp, how="left", predicate="within")

#No of rows in the data
total_births = births_with_geography.groupby("name")["KESSNER"].count().reset_index()
total_births.columns = ["name", "total_births"]

# Filter for births where kesser == 1
births_kesser_1 = births_with_geography[births_with_geography["KESSNER"] == 1]

# Aggregate count of kesser = 1 by geographic area (e.g., 'zone' column from shapefile)
adequate_care_counts = births_kesser_1.groupby("name")["KESSNER"].count().reset_index()
adequate_care_counts.columns = ["name", "kessner_1_count"]

births_summary = total_births.merge(adequate_care_counts, on="name", how="left")
births_summary["percentage_adequate"] = (births_summary["kessner_1_count"] / births_summary["total_births"]) * 100

display(births_summary)

,name,total_births,kessner_1_count,percentage_adequate
0,Detroit,7426,4594,61.863722


### NVI Zones

In [16]:
nvi_shp.set_crs(epsg=4326, inplace=True)

# Ensure both GeoDataFrames have the same CRS
births_gdf = births_gdf.to_crs(nvi_shp.crs)

# Spatial join: Assign each point to a polygon
merged_gdf = gpd.sjoin(births_gdf, nvi_shp, how="left", predicate="intersects")
print(merged_gdf)

#No of rows in the data
total_births_nvi = merged_gdf.groupby("DistrictNu")["KESSNER"].count().reset_index()
total_births_nvi.columns = ["DistrictNu", "total_births"]

# Filter for births where kesser == 1
births_kesser_1_nvi = merged_gdf[merged_gdf["KESSNER"] == 1]

# Aggregate count of kesser = 1 by geographic area (e.g., 'zone' column from shapefile)
adequate_care_counts_nvi = births_kesser_1_nvi.groupby("DistrictNu")["KESSNER"].count().reset_index()
adequate_care_counts_nvi.columns = ["DistrictNu", "kessner_1_count"]

births_summary_nvi = total_births_nvi.merge(adequate_care_counts_nvi, on="DistrictNu", how="left")
births_summary_nvi["percentage_adequate"] = (births_summary_nvi["kessner_1_count"] / births_summary_nvi["total_births"]) * 100

display(births_summary_nvi)

      OCCST  OCCCO  OCCMCD  RESST  RESCO  RESMCD           CENSUS  HOSPITAL  \
0       023     33  6000.0     23     33     400  260650007001023         1   
1       023     63  4900.0     23     63    1300  261251668001012        19   
2       023     63  4900.0     23     82    2200  261635695001012        19   
3       023     82  8900.0     23     82    1600  261635798002006       524   
4       023     82  8900.0     23     82    7700  261635837002006       524   
...     ...    ...     ...    ...    ...     ...              ...       ...   
99174   049     99  9999.0     23     46     200                0       999   
99175   049     99  9999.0     23     82    1400                0       999   
99176   049     99  9999.0     23     52    5352                0       999   
99177    YC     99  9999.0     23     24     400                0       999   
99178    YC     99  9999.0     23     82    1400                0       999   

       BXYEAR  BXMONTH  ...  LONGITUDE             

,DistrictNu,total_births,kessner_1_count,percentage_adequate


In [9]:
point_in_poly_count = sum(1 for point in births_gdf.geometry for poly in nvi_shp.geometry if point.intersects(poly))
print(f"Points intersecting with polygons: {point_in_poly_count} out of {len(births_gdf)}")

Points intersecting with polygons: 0 out of 99179


### Council Districts 2026

In [ ]:
# Ensure both GeoDataFrames have the same CRS
births_gdf_cd = births_gdf.to_crs(cd_shp.crs)

# Perform a spatial join to assign births to geographic areas
births_with_cd = gpd.sjoin(births_gdf_cd, cd_shp, how="left", predicate="within")

#No of rows in the data
total_births_cd = births_with_cd.groupby("council_di")["KESSNER"].count().reset_index()
total_births_cd.columns = ["council_di", "total_births"]

# Filter for births where kesser == 1
births_kesser_1_cd = births_with_cd[births_with_cd["KESSNER"] == 1]

# Aggregate count of kesser = 1 by geographic area (e.g., 'zone' column from shapefile)
adequate_care_counts_cd = births_kesser_1_cd.groupby("council_di")["KESSNER"].count().reset_index()
adequate_care_counts_cd.columns = ["council_di", "kessner_1_count"]

births_summary_cd = total_births_cd.merge(adequate_care_counts_cd, on="council_di", how="left")
births_summary_cd["percentage_adequate"] = (births_summary_cd["kessner_1_count"] / births_summary_cd["total_births"]) * 100

display(births_summary_cd)

Index(['OCCST', 'OCCCO', 'OCCMCD', 'RESST', 'RESCO', 'RESMCD', 'CENSUS',
       'HOSPITAL', 'BXYEAR', 'BXMONTH', 'PLURALITY', 'POUNDS', 'OUNCES',
       'GRAMS', 'ATTENDANT', 'MOMAGE', 'MOMEDUC', 'NOWLIVING', 'NOWDEAD',
       'BORNDEAD', 'LASTLVBXYR', 'LASTLVBXMO', 'MONTHCAREBEG', 'PRENCARE',
       'FRSTVISMO', 'FRSTVISDY', 'FRSTVISYR', 'LSTVISMO', 'LSTVISDY',
       'LSTVISYR', 'PRENVISIT', 'CALCGEST', 'DADAGE', 'DADEDUC', 'ESTWKSGEST',
       'MARITALSTATUS', 'MOMZIP', 'PAYSOURCE', 'BXPLACE', 'SmokedEver',
       'QUITSMOKE', 'QUITMO', 'QUITDAY', 'QUITYR', 'OTHERSMOKE', 'MOMHISP',
       'FATHHISP', 'RF_ALCOHOL', 'WIC', 'BRIDGEMOMRACE', 'BRIDGEDADRACE',
       'NAMEDPARENT', 'SMOKEDPREG', 'SMOK3MOPP', 'SMOK1STTRIM', 'SMOK2NDTRIM',
       'SMOK3RDTRIM', 'BMI', 'KESSNER', 'KOTELCHUCK', 'LATITUDE', 'LONGITUDE',
       'geometry', 'index_right', 'council_di', 'district_n', 'district_1',
       'legal_desc', 'ObjectId', 'Shape__Are', 'Shape__Len'],
      dtype='object')


,council_di,total_births,kessner_1_count,percentage_adequate
0,1.0,958,625,65.240084
1,2.0,926,605,65.334773
2,3.0,1266,775,61.216430
3,4.0,1042,570,54.702495
4,5.0,821,523,63.702801
5,6.0,1097,650,59.252507
6,7.0,1316,846,64.285714
